In [ ]:
# Basic imports to carry out numerical analysis and visualisation:
import pandas
import numpy
import matplotlib.pyplot as plt # for crude plots without annotations just for my own checking not publication grade
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.io as pio
import glob
plotly.offline.init_notebook_mode()
from analysis_functions import boxplot_grouped, grouped_plotter

# Description

The Department for Big Lorries (DfBL) is responsible for ensuring that the goods vehicles used on the nation’s roads do not pose a danger to their passengers, and to other road users. Every year since 2000 the DfBL has conducted inspections on a sample of road-going vehicles. 

The DfBL does not inspect the same vehicles every year, but instead chooses a sample of the vehicles on the road. As a rule, the DfBL aim to inspect each vehicle on the road at least once every 12 years, and inspect most once every 6 years. The DfBL inspects vehicles previously found to be in poor condition more often, and inspects those previously found to be in good condition less often. 

The inspection categorises the vehicles into three types (heavy goods vehicles, light goods vehicles, and personnel), into the ten most common manufacturers, and then grades the vehicles on a 100-point scale, where 100 is perfect and 0 represents a vehicle in hazardous condition.

The attached spreadsheet (VehicleData.xls) contains the results of the inspections from 2000 onwards. The fields in the data are as follows:

    1. The unique ID of the vehicle;
    2. The type of vehicle;
    3. The manufacturer;
    4. The inspection score;
The client hands you this data and asks you to investigate the following questions:

    1. How does the sample size vary from year-to-year? Are there any years that the DfBL should exclude from this analysis?
    2. Assuming that the data is a representative sample of the population; does the condition of road-going vehicles appear to be changing over time? If so, is it deteriorating or improving?
    3. Are there any particular vehicle types or manufacturers that the DfBL should pay special attention to? Would you suggest any changes to the sampling methodology to obtain better information for these types?
    4. Since 2000 the vehicle inspectors have slowly improved the algorithm that they use to determine which vehicles to inspect more frequently, so that they are getting better at inspecting vehicles in poor condition more often. What effect could this have on the results?
    5. Is DfBL conducting a sufficient number of inspections a year? Explain and justify your answer.


## Data imports

Data is read from flat files and excel sheets into pandas DataFrames that are concatenated into one for further analysis in this exercise. Before specific questions are tackled some exploratory analysis is carried out.

In [ ]:
# For a quick import paths are manually added:

file1 = pandas.read_csv(r'./Data/VehicleData_csv.csv')
file2 = pandas.read_excel(r'./Data/VehicleData_2003.xls')
file3 = pandas.read_excel(r'./Data/VehicleData_2010.xlsx')
file = pandas.concat([file1, file2,file3])
file.head()

In [ ]:
# Any null values?:

null_exist = file.isnull().values.any()
print('Do nan values exist: ', null_exist)

string_vals = type(file[['Manufacturer','FinancialYear', 'VehicleType']].values.any())!=str
print('Do wrong strings exist: ', string_vals)

print('Total number of records: ', file.shape[0])

In [ ]:
# Total number of unique vehicles sampled during the whole 12 year period (population size):
vehicle_count = file.groupby('VehicleID').count().iloc[0:-1, 1].shape[0]
print('Total number of unique vehicles sampled: ', vehicle_count)

# the frequency of most and least checked vehicles:
min_checked_count = file.groupby('VehicleID').size().min()
max_checked_count = file.groupby('VehicleID').size().max()
print(f'min and max check counts are: {min_checked_count} and {max_checked_count}')

## Q1

The size of a population 20 508 (the count of unique vehicle ids in the whole dataframe) because it was said that every vehicle is inspected at least once every 12 years and investion report covers 12 years. We do not know that is a score distribution of population. Our sampling is biased and sample distribution is not the same as population distribution. We are heavily biased to take sample from the low score tail of population distribution.

To check that most vehicles are checked at least every 6 years we should get at least 2 records per vehicle id for most vehicle id's. Indeed the minimum amount each vehicle was checked is 3 amd maximum is 24. 

In [ ]:
# Constructing a Dataframe for annual sample size, exponentially weighted mean sample
# size (where damping is eqiuivalent of 3 years window) and exponentially weighted
# standard deviation of the same window size~

count_stats = file.groupby('FinancialYear').size().reset_index()
count_stats.columns = ['FinancialYear', 'SampleSize']
count_stats['exp_weighed_mean_sample_size'] = count_stats.SampleSize.ewm(span=3, adjust=False).mean()
count_stats['exp_weighed_std_sample_size'] = count_stats.SampleSize.ewm(span=3, adjust=False).std()

count_stats

In [ ]:
# Plot annual sample size as a scatter plot, also +1/-1 STD around it as a shaded region
# and an exponentially weighted rolling mean as a thick line:
# Plotting in this cell is not wrapped under a function as I
# hardly see it can be reused in the same fashion

upper_bound = go.Scatter(
    name='Exponentially weighted standard deviation',
    x=count_stats.FinancialYear,
    y=count_stats.SampleSize+count_stats.exp_weighed_std_sample_size,
    mode = 'lines',
    line=dict(width=0.5,
              color='rgb(131, 90, 241)'),
    fill='tonexty')
    

lower_bound = go.Scatter(
    name='Lower Bound',
    x=count_stats.FinancialYear,
    y=count_stats.SampleSize-count_stats.exp_weighed_std_sample_size,
    showlegend=False,
    mode = 'lines',
    line=dict(width=0.5,
              color='rgb(131, 90, 241)'))

trace = go.Scatter(
    name='Actual sample size',
    x=count_stats.FinancialYear,
    y=count_stats.SampleSize,
    showlegend=True,
    mode = 'markers')

trace2 = go.Scatter(
    name='Exponentially weighted rolling mean',
    x=count_stats.FinancialYear,
    y=count_stats.exp_weighed_mean_sample_size,
    mode = 'lines',
    line=dict(width=1.5,
              color='navy'))


data = [trace, lower_bound, upper_bound, trace2]

layout= go.Layout(
    title= None,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Financial year',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Count (number of vehicles)',
        ticklen= 5,
        gridwidth= 2,
    ),
    legend=dict(orientation="h",
               x=0, y=-0.3)
    )
fig= go.Figure(data=data, layout=layout)
iplot(fig)

# Export this plot as a vector graphic image:
pio.write_image(fig, './images/sample_size_variation.svg')

We see that sample size in year 2000 is 4.3 times smaller than the upcoming year. Taking sample sizes for the 12 year period into consideration this clearly is a candidate for an outlier. Yet can it be used for further analysis? We should not use the year 2000 data, because it does not generalise to the population statistics the same way the other year values do and we can therefore not compare insights drawn in 2000 to subsequent years.

In [ ]:
# Removing records from the year 2000-2001:

file = file[file.FinancialYear!='2000-01']

## Q2


First let's check a very crude trend of overall ConditionScore each year, where a bigger value indicates greater condition. We see that this crude way shows that each year vehicles score from 44 to 52. The spread is very similar both in terms of STD or MAD and we can not yet say anything about the shape of the distribution. Maybe I'll have to histogram it for a few years chosen see how they compare in shape. 

In [ ]:
# Check what is the overall condition score and mean absolute deviation
# each year and assign it to the variable "annual_condition":

annual_condition = file.groupby('FinancialYear').ConditionScore.agg(['mean', 'mad','std']).reset_index()
annual_condition.columns = ['FinancialYear', 'mean_score', 'mad_score', 'std_score']
annual_condition

In [ ]:
# Quickly check how a boxplot of means where all vehicle types are binned
# togeter looks like:

file.boxplot(column='ConditionScore', by='FinancialYear', rot=45, figsize=(10,10))

It looks that boxplotting records with really large 1-3 interquartile spread range spoils the visual effect of median (default pandas setting for green line) score value trend throughtout years. There is no particular need to use STD, because we can not infer confidence intervals: condition score values are not supposed to be normally distributed per each year. Therefore MAD can equally be used as it is less prone to be biased by outliers (L1 vs L2 type of loss). Let's make a nicer plotly figure with a boxploted scores for each year but grouped by vehicle type. This will illustrate trend in both mean value and the spread of values around it. 

In [ ]:
# Constructing a dataframe "mean_ConditionScore" that summarizes mean score for each
# type of vehicle every year:

pivot_ConditionScore = file.pivot_table(values='ConditionScore', index='FinancialYear', columns='VehicleType', aggfunc=['mean','mad']).reset_index()
pivot_ConditionScore

In [ ]:
# Boxplot each year's score grouped by vehicle type using imported function where I factored out some
# functions I defined myself:

colors = ['rgb(17,30,108)', 'rgb(15,82,186)', 'rgb(137,207,240)']

plot = boxplot_grouped(
    dataframe=file,
    x_col='FinancialYear',
    y_col='ConditionScore',
    group_col='VehicleType',
    colors=colors,
    title=None,
    x_title='Financial year',
    y_title='Condition score'
    )

# Export this plot as a vector graphic image:
pio.write_image(plot, './images/condition_score_variation_boxplot.svg')

In [ ]:
# Also plot just the variation of mean. Dispersion illustration is
# missing in it but the trend is a lot clearer to follow:

x = pivot_ConditionScore.FinancialYear
trace1 = go.Scatter(
    name='Heavy Goods Vehicle',
    x=x,
    y=pivot_ConditionScore['mean']['Heavy Goods Vehicle'],
    mode = 'markers',
    showlegend=True,
    marker=dict(
        color=colors[0],
        size=10,
        line=dict(width=1.5)))

trace2 = go.Scatter(
    name='Light Goods Vehicle',
    x=x,
    y=pivot_ConditionScore['mean']['Light Goods Vehicle'],
    mode = 'markers',
    showlegend=True,
    marker=dict(
        color=colors[1],
        size=10,
        line=dict(width=1.5)))

trace3 = go.Scatter(
    name='Personnel Vehicle',
    x=x,
    y=pivot_ConditionScore['mean']['Personnel Vehicle'],
    mode = 'markers',
    showlegend=True,
   marker=dict(
        color=colors[2],
        size=10,
        line=dict(width=1.5)))

data = [trace1, trace2, trace3]

layout= go.Layout(
    title= None,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Financial year',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Average annual condition score',
        ticklen= 5,
        gridwidth= 2,
    ),
    legend=dict(orientation="h",
               x=0, y=-0.3)
    )
fig= go.Figure(data=data, layout=layout)
iplot(fig)

# Export this plot as a vector graphic image:
pio.write_image(fig, './images/condition_score_variation_means.svg')

## Q3

Lets get the total condition score per manufacturer per financial year in each vehicle category. We know that heavy and light goods vehicles consistently score worse than personnel vehicles thus binning them all together would not be fair. 

In [ ]:
# Calculating mean condition score divided by the amount of inspected
# vehicles every year in each vehicle tyope category:

av_annual_score_each_manufacturer = file.groupby(['FinancialYear', 'VehicleType','Manufacturer']).agg({'ConditionScore':sum, 'VehicleID':'count'}).reset_index()
av_annual_score_each_manufacturer.columns = ['FinancialYear', 'VehicleType', 'Manufacturer', 'TotalScore', 'VehicleCount']
av_annual_score_each_manufacturer['ConditionScorePerVehicle'] = av_annual_score_each_manufacturer.TotalScore/av_annual_score_each_manufacturer.VehicleCount

# Inspecting a few top records:

av_annual_score_each_manufacturer.head(7)

In [ ]:
# Plotting the dynamics of each manufacturers total annual score for each vehicle type:

fig = grouped_plotter(
    av_annual_score_each_manufacturer,
    plot_title='Total annual score for each manufacturer',
    kwargs=None)
# Export this plot as a vector graphic image:
pio.write_image(fig, './images/total_annual_score_each_manufacturer.svg')

In [ ]:
# Plotting the dynamics of each manufacturers annual
# score per vehicle (total score divided by the amount of vehicles inspected) for each vehicle type:

fig = grouped_plotter(
    av_annual_score_each_manufacturer,
    plot_title='Annual score per vehicle for each manufacturer',
    kwargs={
        
        'x': 'FinancialYear',
        'x_title':'Financial year',
        'y': 'ConditionScorePerVehicle',
        'y_title':'Annual score per vehicle',
        'group_column_inner': 'Manufacturer',
        'group_column_outer': 'VehicleType'})
# Export this plot as a vector graphic image:
pio.write_image(fig, './images/annual_score_per_vehicle_each_manufacturer.svg')

## Q4

This answer will be answered without the need to use computing.

## Q5

To answer this question we need to investigate the standard error of sample mean and confidence intervals for our records. We know that the larger the sample size the closer is the sample estimator to the population estimator being that mean, variance or any other parameter.

The question we have to ask is how precise do we have to know the value of records. This precision value has to be significantly smaller than the trend we are trying to illustrate. If we are judging which car manufacturer is better and worse based on each of their mean annual score we have to know that the standard error of mean is smaller than the difference between mean scores for those manufacturers.

To be able to tell if the sample size is large enough I will compare the differences in annual score values per vehicle category (scatter graph for Q2) against the standard error of sample means for a corresponding year. If it is a small proportion of the corresponding difference then it means it is precise enough to make a valid comparison and increasing sample size would not benefit analysis.

To evaluate the standard error of sample mean and standard error of sample standard deviation of an unknown distribution (our score distributions are not normal for sure, see a few quick histogram plots underneath) I will use non parametric bootstrap to build up a histogram of resampled means and its standard deviation is the value of sample's standar error.

In [ ]:
# Score distribution of Personnel Vehicles in 2001 and 2002 (score and its frequency) for all manufacturers:

for i in file.FinancialYear.unique()[0:2]:
    file[(file.VehicleType=='Personnel Vehicle')&(file.FinancialYear== i)].ConditionScore.hist(bins=80)

In [ ]:
# Score distribution of Light Goods Vehicles in 2001 and 2002  (score and its frequency) for all manufacturers:

for i in file.FinancialYear.unique()[0:2]:
    file[(file.VehicleType=='Light Goods Vehicle')&(file.FinancialYear== i)].ConditionScore.hist(bins=80)

In [ ]:
# Score distribution of Heavy Goods Vehicles in 2001 and 2002 (score and its frequency) for all manufacturers:

for i in file.FinancialYear.unique()[0:2]:
    file[(file.VehicleType=='Heavy Goods Vehicle')&(file.FinancialYear== i)].ConditionScore.hist(bins=80)

In [ ]:
# Below I defined a numpy implementation of bootstrap standard error and confidence interval calculation


def nonparam_bootstrap_st_err(data, repeats):
    """ Returns a bootstrap standard error
        of sample mean for a given dataset.
    """
    
    resampled_set = numpy.random.choice(data,(len(x), repeats))
    distrib_of_means = resampled_set.mean(axis=0)
    return numpy.std(distrib_of_means)

def nonparam_bootstrap_conf_int(data, repeats, conf_interval=95):
    """ Returns a bootstrap confidence interval of a given dataset.
        return type is a float that corresponds to half the range
        of values covered under the selected "conf_interval" so that
        the mean can be quoted sample mean +/-"return of this function". 
    """
    
    resampled_set = numpy.random.choice(data,(len(x), repeats))
    distrib_of_means = resampled_set.mean(axis=0)
    distrib_of_means.sort()
    percentile = numpy.percentile(distrib_of_means, [0.5*(100-conf_interval),100-(0.5*(100-conf_interval))])
    return 0.5*(percentile[1]-percentile[0])

In [ ]:
# create annual_means dataframe and populate extra columns for corresponding
# standard errors of means and confidence intervals (90 % but we can choose the tolerance) 

annual_means = pandas.concat([pivot_ConditionScore['FinancialYear'],pivot_ConditionScore['mean']],axis=1)
annual_means.head()

In [ ]:
# Define a function that takes the originally given big datafile and the one with
# mean scores per each year per vehicle category and creates extra columns. Those
# columns are then filled by values generated by feeding values of the original
# dataframe iteratively 

def SE_calculator(df1, df2, repeats, func, conf_interval=95):
    df1 = df1.copy()
    max_ind = df1.shape[1]
    df1['HGV_st_err'] = numpy.NaN
    df1['LGV_st_err'] = numpy.NaN
    df1['PV_st_err'] = numpy.NaN
    for i in df1.index:
        for j,k in zip(df1.columns[1:max_ind], df1.columns[max_ind:]):
            x = df2[(df2.VehicleType==j)&
                 (df2.FinancialYear== df1.loc[i, 'FinancialYear'])].ConditionScore.values
            df1.at[i, k] =func(x, repeats)
    return df1

In [ ]:
# Apply function SE_calculator on annual_means dataframe using data in files dataframe:

st_error_df = SE_calculator(annual_means, file, repeats=10000, func=nonparam_bootstrap_st_err)
st_error_df = st_error_df.round(3)

## Check sampling

In [ ]:
# Visually inspectiong proportions of each vehicle in a category every year:

proportions = file.groupby(['FinancialYear', 'VehicleType']).agg({'VehicleID':'count'}).reset_index()
proportions = proportions.pivot_table(values='VehicleID', index='FinancialYear', columns='VehicleType').reset_index()
proportions

## Some calculations and plots for written task (government spending)

In [ ]:
# Read a csv file into pandas frame:

spend_df = pandas.read_csv('./Data/gov_spend.csv')
spend_df.head() # just to look at the top rows

In [ ]:
# Calculating differences in transport
# infrastructure ecpenditures in successive years:

spend_df['diff_transport_spend'] = spend_df.transport_spend.diff()
print('average spending increase: ', str(spend_df.diff_transport_spend.mean()))

# Calculate percentage of transport expenses in total spent amount (in %):
spend_df['ratio_transport_spend'] = 100*(spend_df.transport_spend/spend_df.total_spend)
print('average spending proportion to total spent: ', str(spend_df.ratio_transport_spend.mean()))
spend_df

In [ ]:
# Plotting spending on Transport and population vs time:

trace1 = plotly.graph_objs.Scatter(
    x=spend_df.Year,
    y=spend_df.population,
    name='UK population'
)
trace2 = plotly.graph_objs.Scatter(
    x=spend_df.Year,
    y=spend_df.transport_spend,
    name='Government spending on transport infrastructure'
)

data = [trace2,trace1]
layout= plotly.graph_objs.Layout(
    title= None,
    showlegend=True,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Financial year',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Expenditure bn (GBP)',
        ticklen= 5,
        gridwidth= 2,
    ),
    legend=dict(orientation="h",
               x=0.2, y=-0.2),
        )

fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='stacked-bar')

plotly.io.write_image(fig, './images/transport_spending.svg')

In [ ]:
# Plotting spending percentage on Transport vs time:

trace1 = plotly.graph_objs.Scatter(
    x=spend_df.Year,
    y=spend_df.ratio_transport_spend,
    name='UK population'
)

layout= plotly.graph_objs.Layout(
    title= None,
    showlegend=False,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Financial year',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Proportion of transport expenditure to total (%)',
        ticklen= 5,
        gridwidth= 2,
    ),
    legend=dict(orientation="h",
               x=0.2, y=-0.2),
        )

fig2 = plotly.graph_objs.Figure(data=[trace1], layout=layout)
plotly.offline.iplot(fig2, filename='stacked-bar')

plotly.io.write_image(fig2, './images/transport_spending_percentage.svg')